In [ ]:
"""
Module: BLASTp-Based Autoimmunity Screening
File: Blastp_com.ipynb
Author: Milind Shrivastava (M.Sc. Biotechnology)

Purpose:
Automates BLASTp short-query searches for candidate peptide epitopes
to identify host-homology and reduce autoimmunity risk prior to
HLA epitope prediction.

Pipeline Context:
Protein retrieval → Conserved region analysis → Peptide generation →
Autoimmunity screening (this module) → HLA Class I/II prediction →
Vaccine construct design.

Key Features:
• Batch BLASTp execution for short peptides (<30 aa)
• Short-query optimized BLASTp configuration
• Structured outputs for downstream Excel/VBA-based filtering
• Lightweight logging and error handling

Note:
Developed during MSc thesis work and later refined for clarity and
reproducibility without altering biological logic.
"""

In [2]:
import time
import random
from concurrent.futures import ThreadPoolExecutor
from Bio.Blast import NCBIWWW, NCBIXML
from Bio import SeqIO
import pandas as pd
import os

def run_blastp_for_record(record, organism_filter="Homo sapiens", retries=3):
    seq_len = len(record.seq)
    seq_id = record.id
    print(f"🔬 Submitting sequence {seq_id} (length: {seq_len})...")

    is_short = seq_len <= 30

    for attempt in range(1, retries + 1):
        try:
            if is_short:
                print(f"⚙️  Using short_query=True for {seq_id}")
                result_handle = NCBIWWW.qblast(
                    program="blastp",
                    database="nr",
                    sequence=str(record.seq),
                    entrez_query=f"{organism_filter}[Organism]",
                    format_type="XML",
                    short_query=True
                )
            else:
                print(f"⚙️  Using standard BLAST settings for {seq_id}")
                result_handle = NCBIWWW.qblast(
                    program="blastp",
                    database="nr",
                    sequence=str(record.seq),
                    entrez_query=f"{organism_filter}[Organism]",
                    format_type="XML",
                    expect=0.05,
                    word_size=6
                )

            return seq_id, result_handle.read()
        except Exception as e:
            print(f"⚠️ Attempt {attempt} failed for {seq_id}: {e}")
            time.sleep(random.uniform(5, 10))

    print(f"❌ Failed all attempts for {seq_id}")
    return seq_id, None

def parse_xml_string(xml_string, query_id):
    from io import StringIO
    blast_record = NCBIXML.read(StringIO(xml_string))

    hits = []
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            hits.append({
                "Query ID": query_id,
                "Accession": alignment.accession,
                "Title": alignment.title,
                "Length": alignment.length,
                "Score": hsp.score,
                "E-value": hsp.expect,
                "Identities": hsp.identities,
                "Alignment Length": hsp.align_length,
                "Query": hsp.query,
                "Match": hsp.match,
                "Subject": hsp.sbjct
            })
    return hits

def blast(fasta_file, output_excel="Seq1_results.xlsx", organism="Homo sapiens", max_workers=3):
    print("🚀 Starting parallel BLASTp runs...")
    records = list(SeqIO.parse(fasta_file, "fasta"))
    all_hits = []

    def process_and_parse(record):
        seq_id, xml_string = run_blastp_for_record(record, organism)
        if xml_string:
            hits = parse_xml_string(xml_string, seq_id)
            return hits
        return []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = executor.map(process_and_parse, records)

    for hits in results:
        all_hits.extend(hits)

    if all_hits:
        df = pd.DataFrame(all_hits)
        df.to_excel(output_excel, index=False)
        print(f"📊 Combined BLAST results saved to {output_excel}")
    else:
        print("⚠️ No successful BLAST results to save.")

# === USAGE ===
fasta_input = "Seq1_2.fasta"
output="Seq1_2_results.xlsx"
blast(fasta_input, output_excel = output)

fasta_input = "Seq2.fasta"
output="Seq2_results.xlsx"
blast(fasta_input, output_excel = output)

fasta_input = "Seq3.fasta"
output="Seq3_results.xlsx"
blast(fasta_input, output_excel = output)

🚀 Starting parallel BLASTp runs...
🔬 Submitting sequence Seq1 (length: 15)...
⚙️  Using short_query=True for Seq1
🔬 Submitting sequence Seq2 (length: 15)...
⚙️  Using short_query=True for Seq2
🔬 Submitting sequence Seq3 (length: 15)...
⚙️  Using short_query=True for Seq3
🔬 Submitting sequence Seq4 (length: 15)...
⚙️  Using short_query=True for Seq4
🔬 Submitting sequence Seq5 (length: 15)...
⚙️  Using short_query=True for Seq5
🔬 Submitting sequence Seq6 (length: 15)...
⚙️  Using short_query=True for Seq6
🔬 Submitting sequence Seq7 (length: 15)...
⚙️  Using short_query=True for Seq7
📊 Combined BLAST results saved to Seq1_2_results.xlsx
🚀 Starting parallel BLASTp runs...
🔬 Submitting sequence Seq1 (length: 15)...
⚙️  Using short_query=True for Seq1
🔬 Submitting sequence Seq2 (length: 15)...
⚙️  Using short_query=True for Seq2
🔬 Submitting sequence Seq3 (length: 15)...
⚙️  Using short_query=True for Seq3
🔬 Submitting sequence Seq4 (length: 15)...
⚙️  Using short_query=True for Seq4
🔬 Submi